## Response Evaluation

In [12]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import FaithfulnessEvaluator

In [13]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("D:\Data\LLM\llamaindex").load_data()

In [6]:
#documents

In [15]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [16]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)

In [41]:
# create llm
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.0)

In [18]:
evaluator = FaithfulnessEvaluator(llm=llm)

In [20]:
query_engine = index.as_query_engine()

In [21]:
response = query_engine.query("What are the Key Sectors of the Indian Economy?")
print(response)

The key sectors of the Indian economy are agriculture, industry, and services.


In [27]:
import nest_asyncio
nest_asyncio.apply()

In [26]:
query = "What are the Key Sectors of the Indian Economy?"
response = query_engine.query(query)
eval_result = evaluator.evaluate_response(query=query, response=response)
print(str(eval_result))

query='What are the Key Sectors of the Indian Economy?' contexts=["Indian Economy: An Overview\nKey Sectors of the Indian Economy\nAgriculture: Agriculture has historically been the backbone of the Indian economy. It employs around\n42% of the workforce and contributes approximately 16% to the GDP. Key agricultural products\ninclude rice, wheat, pulses, spices, and cotton. Despite its importance, the sector faces challenges\nsuch as low productivity, fragmented land holdings, and dependence on monsoons.\nIndustry: The industrial sector has witnessed significant growth post-reforms. It contributes about\n25% to the GDP and includes sub-sectors like manufacturing, mining, and construction. The 'Make\nin India' initiative launched in 2014 aims to boost manufacturing, attract FDI, and create jobs,\npositioning India as a global manufacturing hub.\nServices: The services sector is the largest contributor to the Indian economy, accounting for around\n55% of the GDP. It encompasses a wide ran

In [28]:
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

True


In [29]:
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.feedback))

YES


## Retrieval Evaluation

In [32]:
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.core.retrievers import VectorIndexRetriever

In [37]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

In [38]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)


In [39]:
retriever_evaluator.evaluate(
    query="query", expected_ids=["node_id1", "node_id2"]
)

RetrievalEvalResult(query='query', expected_ids=['node_id1', 'node_id2'], expected_texts=None, retrieved_ids=['cdb25d78-aa75-4d4f-820e-b2cca53f1487', 'c7722bf7-32bd-4a2f-849b-c185708e800c'], retrieved_texts=['Indian Economy: An Overview\nIndian Economy: An Overview\nPage 1', "Indian Economy: An Overview\nHistorical Background\nThe economic history of India is a story of contrast and transformation. Pre-independence, India was\none of the world's richest economies, heavily relying on agriculture and trade. However, colonial\nexploitation and subsequent policies led to a significant decline in economic stature.\nPost-independence in 1947, India adopted a mixed economy model, combining features of both\ncapitalism and socialism. The government focused on self-reliance, implementing policies of import\nsubstitution industrialization. The economy, however, was hampered by inefficiencies and slow\ngrowth, often referred to as the 'Hindu rate of growth.'\nPage 3"], mode=<RetrievalEvalMode.TEX

## Cost Analysis

In [42]:
from llama_index.core.llms import MockLLM
from llama_index.core import MockEmbedding

llm = MockLLM(max_tokens=256)
embed_model = MockEmbedding(embed_dim=1536)

In [43]:
import tiktoken
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler

In [44]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

In [45]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.callback_manager = callback_manager

In [46]:
print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

Embedding Tokens:  0 
 LLM Prompt Tokens:  0 
 LLM Completion Tokens:  0 
 Total LLM Token Count:  0 



In [47]:
token_counter.reset_counts()

In [49]:
query_engine = index.as_query_engine()

response = query_engine.query("What are the Key Sectors of the Indian Economy?")

print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

Embedding Tokens:  0 
 LLM Prompt Tokens:  445 
 LLM Completion Tokens:  256 
 Total LLM Token Count:  701 

